## Package Imports

In [1]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import glob

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, clear_output
from functools import reduce
import math
import warnings

warnings.simplefilter("ignore")

## Read in Data

In [2]:
# Transform csv for modeling
def load_indicators(indicator_filenames):
    """
    indicator_filenames (str) : indicator filenames separated by a spaces
    
    Returns
    --
    merged_indicators (pd.DataFrame) : indicator
    """
    
    # Case where we use a single indicator
    if len(indicator_filenames) == 1:
        print("Loading one indicator")
        if ".csv" not in indicator_filenames:
            indicator_filenames += ".csv"
            
        indicator = pd.read_csv("Data_Copy/Economic indicators/" + indicator_filenames)
        indicator["Date"] = pd.to_datetime(indicator['DateTime']).dt.date
        return indicator[["Date", "Value"]]
    
    # Case where we use multiple indicators
    print("Loading multiple indicators")
    # Add .csv if not at the end
    for i, ind_file in enumerate(indicator_filenames):
        if ".csv" not in ind_file:
            indicator_filenames[i] += ".csv"
            
    # read in indicator files
    indicators1 = [pd.read_csv("Data_Copy/Economic indicators/" + ind_file)
                  for ind_file in indicator_filenames]
    
    # Turn DateTime string into Date objects
    indicators2 = []
    for indicator in indicators1:
        indicator["Date"] = pd.to_datetime(indicator["DateTime"]).dt.date
        indicators2.append(indicator[["Date", "Value"]])
        
    start_dates = [ind["Date"].min() for ind in indicators2]
    start_date = max(start_dates)
        
    # remove dates before last start date
    indicators3 = []
    ind3idx = []
    # Keep dates that are in every indicator file being used
    for i, ind in enumerate(indicators2):
        ind3 = ind[ind["Date"] >= start_date]
        indicators3.append(ind3)

    # Source: https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
    merged_indicators = \
        reduce(lambda left,right: pd.merge(left, right,
                                           on=['Date'], how='outer'),
                       indicators3).sort_values(by="Date").reset_index(drop=True)
    columns = merged_indicators.columns
    nearest_val_dict = {"nearest_" + col : None for col in columns}
    for idx, row in merged_indicators.iterrows():
        for col in columns:
            if "Date" not in col:
                if math.isnan(row[col]):
                    if nearest_val_dict["nearest_" + col] is None:
                        for val in merged_indicators[col]:
                            if not math.isnan(val):
                                nearest_val_dict["nearest_" + col] = val
                                break
                    merged_indicators[col].iloc[idx] =  nearest_val_dict["nearest_" + col]
                else:
                    nearest_val_dict["nearest_" + col] = row[col]
    v = 0
    rename_cols = {}
    for col in columns:
        if "Date" in col:
            continue
        v += 1
        rename_cols[col] = "Value" + str(v)
        ""
    return merged_indicators.rename(columns=rename_cols)

In [3]:
def merge_indicators_sales(data, indicators):
    startDate = max([data["Date"].min(), indicators["Date"].min()])
    data = data[data["Date"]>startDate].set_index("Date")
    indicators = indicators[indicators["Date"]>startDate].set_index("Date")
    merged = pd.concat([indicators, data], axis=1)
    # Fill NAs
    columns = merged.columns
    nearest_val_dict = {"nearest_" + col : None for col in columns}
    for idx, row in merged.iterrows():
        for col in columns:
            if "Date" not in col:
                if math.isnan(row[col]):
                    if nearest_val_dict["nearest_" + col] is None:
                        for val in merged[col]:
                            if not math.isnan(val):
                                nearest_val_dict["nearest_" + col] = val
                                break
                    merged[col].loc[idx] =  nearest_val_dict["nearest_" + col]
                else:
                    nearest_val_dict["nearest_" + col] = row[col]
    # split data into day, month, year
    return merged.reset_index()    

In [12]:
ind_files = glob.glob("Data_Copy/Economic Indicators/*.csv")
ind_files = [i[73:] for i in ind_files]

In [20]:
ind_files = glob.glob("Data_Copy/Economic Indicators/*.csv")
ind_names = [i[73:] for i in ind_files]
print("Choose one or multiple indicators.")
w = widgets.SelectMultiple(
    options=ind_names,
    description='Options:',
    disable=False,
    layout=Layout(width='30%', height='300px')
)
display(w)

Choose one or multiple indicators.


SelectMultiple(description='Options:', layout=Layout(height='300px', width='30%'), options=('API_Product_Impor…

In [22]:
ind_files_used = w.value
start = "historical_country_United_States_indicator_"
indicators = load_indicators([start+str(x) for x in ind_files_used])
indicators

Loading multiple indicators


,Date,Value1,Value2,Value3
0,2001-11-16,-760.0,58081.0,2.8
1,2001-11-23,-692.0,58081.0,2.8
2,2001-11-30,1669.0,58081.0,2.8
3,2001-12-07,-890.0,58081.0,2.8
4,2001-12-14,-194.0,58081.0,2.8
...,...,...,...,...
1227,2021-07-16,2438.0,110850.0,4.5
1228,2021-07-23,-616.0,110850.0,4.5
1229,2021-07-30,510.0,110850.0,4.5
1230,2021-07-31,510.0,110850.0,4.3


In [23]:
data = pd.read_csv('Data_Copy/Costco_Monthly Sales from 2012 to 2021.csv')
data = data.rename(columns={"Date":"ds","Net Sales (billions)":"y"}).dropna()

date = []
for idx, row in data.iterrows():
    m = str(row["Month"])
    m = "0"+m if len(m)<2 else m
    new_val = str(row["Year"]) + "-" + m + "-01"
    date.append(new_val)
    
data["Date"] = date
data["Date"] = pd.to_datetime(data["Date"]).dt.date
data = data.drop(columns=["ds", "Month", "Year", "Growth Rate"])
data

,y,Date
0,7.00,2012-01-01
2,9.13,2012-03-01
3,7.25,2012-04-01
4,7.67,2012-05-01
5,9.18,2012-06-01
...,...,...
109,14.05,2021-02-01
110,18.21,2021-03-01
111,15.21,2021-04-01
112,15.59,2021-05-01


## Merge Sales and Indicators

In [24]:
df = merge_indicators_sales(data, indicators)
df = df.rename(columns={"Date":"ds"})
df

,ds,Value1,Value2,Value3,y
0,2012-01-06,883.0,20908.0,2.2,9.18
1,2012-01-13,-1643.0,20908.0,2.2,9.18
2,2012-01-20,588.0,20908.0,2.2,9.18
3,2012-01-27,27.0,20908.0,2.2,9.18
4,2012-01-31,27.0,112127.0,2.3,9.18
...,...,...,...,...,...
689,2021-02-01,-796.0,110850.0,4.3,14.05
690,2021-03-01,-796.0,110850.0,4.3,18.21
691,2021-04-01,-796.0,110850.0,4.3,15.21
692,2021-05-01,-796.0,110850.0,4.3,15.59


## Split data into training and testing

In [25]:
df_train = df[df["ds"]<pd.to_datetime("2018-01-01")]
df_test = df[df["ds"]>pd.to_datetime("2018-01-01")]
df_test = df_test[df_test["ds"]<pd.to_datetime("2019-01-01")]
df_train.values.shape, df_test.values.shape

((434, 5), (72, 5))

In [26]:
data = data.rename(columns={"Date":"ds"})
data_train = data[data["ds"] < pd.to_datetime("2018-01-01")]
data_test = data[data["ds"]>pd.to_datetime("2018-01-01")]
data_test = data_test[data_test["ds"]<pd.to_datetime("2019-01-01")]
data_train.values.shape, data_test.values.shape

((71, 2), (11, 2))

## Modeling

In [27]:
m1 = Prophet() # multiple indicators
m2 = Prophet() # no indicators
m3 = Prophet() # original dataset
m1.add_country_holidays(country_name='US')
m2.add_country_holidays(country_name='US')
m3.add_country_holidays(country_name='US')
for col in df_train.columns:
    if "Value" in col:
        m1.add_regressor(col)
        
m1.fit(df_train)
m2.fit(df_train)
m3.fit(data_train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -12.9352
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1019.09    0.00365635       119.528      0.4993      0.4993      112   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     170       1026.57    0.00059307       193.904   2.561e-06       0.001      240  LS failed, Hessian reset 
     199       1027.84   0.000521179       78.6988      0.7233      0.7233      273   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       1029.52    0.00369532       98.5454      0.1572           1      389   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     349       1030.95   0.000437765       138.109   1.893e-06       0.001      503  LS failed, Hessian reset 
     399       1032.58    0.00172706       88.6725      0.2913           1      562   
    Iter      log pro

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -12.9352
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1021.05    0.00548981        127.42           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        1026.1    0.00133549       113.876           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       1027.15     0.0233627       107.081           1           1      354   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     316       1028.49     0.0042342       259.159   4.595e-05       0.001      451  LS failed, Hessian reset 
     392        1030.5    5.2069e-05       94.6616   5.546e-07       0.001      577  LS failed, Hessian reset 
     399        1030.5   7.92111e-06       88.2815      0.3955      0.3955      584   
    Iter      log pro


Initial log joint probability = -125.501
Iteration  1. Log joint probability =    63.8273. Improved by 189.329.
Iteration  2. Log joint probability =     106.73. Improved by 42.9022.
Iteration  3. Log joint probability =    152.812. Improved by 46.0826.
Iteration  4. Log joint probability =    196.039. Improved by 43.227.
Iteration  5. Log joint probability =    228.786. Improved by 32.7465.
Iteration  6. Log joint probability =    228.791. Improved by 0.00532374.
Iteration  7. Log joint probability =    231.442. Improved by 2.65133.
Iteration  8. Log joint probability =    231.485. Improved by 0.0424037.
Iteration  9. Log joint probability =    231.644. Improved by 0.159241.
Iteration 10. Log joint probability =    231.739. Improved by 0.0954077.
Iteration 11. Log joint probability =    231.923. Improved by 0.183339.
Iteration 12. Log joint probability =    231.955. Improved by 0.0320121.
Iteration 13. Log joint probability =    232.062. Improved by 0.107706.
Iteration 14. Log joint 

In [28]:
forecast1 = m1.predict(df_test)
forecast2 = m2.predict(df_test)
forecast3 = m3.predict(data_test)

 28. Log joint probability =    234.329. Improved by 0.018909.
Iteration 29. Log joint probability =    234.698. Improved by 0.369301.
Iteration 30. Log joint probability =    234.703. Improved by 0.00412386.
Iteration 31. Log joint probability =    234.782. Improved by 0.0797545.
Iteration 32. Log joint probability =    235.122. Improved by 0.340098.
Iteration 33. Log joint probability =    235.203. Improved by 0.0803603.
Iteration 34. Log joint probability =    237.518. Improved by 2.31509.
Iteration 35. Log joint probability =    238.233. Improved by 0.714808.
Iteration 36. Log joint probability =    238.609. Improved by 0.376032.
Iteration 37. Log joint probability =     239.27. Improved by 0.661048.
Iteration 38. Log joint probability =    239.404. Improved by 0.134447.
Iteration 39. Log joint probability =    240.065. Improved by 0.660805.
Iteration 40. Log joint probability =    240.203. Improved by 0.137681.
Iteration 41. Log joint probability =    242.462. Improved by 2.25914.

In [29]:
mse1 = mean_squared_error(df_test["y"], forecast1["yhat"], squared=False)
mse2 = mean_squared_error(df_test["y"], forecast2["yhat"], squared=False)
mse3 = mean_squared_error(data_test["y"], forecast3["yhat"], squared=False)
std12 = df['y'].std()
std3 = data['y'].std()

In [30]:
evr1 = (std12 - mse1) / std12
evr2 = (std12 - mse2) / std12
evr3 = (std3 - mse3) / std3
evr1, evr2, evr3

(0.055573846600964205, 0.055381040761057675, 0.9011149376356674)